In [134]:
# ideas

# TODO: analyze erroneous predictions

from datetime import datetime
import math
import random

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [2]:
pd.options.display.max_columns = None

In [3]:
game_states_data_test = pd.read_csv('game_states_data_test.csv')

In [4]:
game_states_data_training = pd.read_csv('game_states_data_training.csv')
game_states_data_training.head()

,instance_id,frag,map_id,game_mode,moving_player,player0_colour,player1_colour,player0_AI,player1_AI,hp_player0_unit1,hp_player0_unit2,hp_player0_unit3,hp_player0_unit4,hp_player1_unit1,hp_player1_unit2,hp_player1_unit3,hp_player1_unit4,position_x_player0_unit1,position_x_player0_unit2,position_x_player0_unit3,position_x_player0_unit4,position_x_player1_unit1,position_x_player1_unit2,position_x_player1_unit3,position_x_player1_unit4,position_y_player0_unit1,position_y_player0_unit2,position_y_player0_unit3,position_y_player0_unit4,position_y_player1_unit1,position_y_player1_unit2,position_y_player1_unit3,position_y_player1_unit4,player0_unit1_type,player0_unit2_type,player0_unit3_type,player0_unit4_type,player1_unit1_type,player1_unit2_type,player1_unit3_type,player1_unit4_type
0,1,True,Map03,Domination,1,#0000FF,#FF0000,False,False,100,36,140,0,20,0,18,0,13.128258,-5.072649,-9.833621,-6.183261,-6.545395,-2.876717,7.301014,2.736246,4.488010,1.017117,3.776405,2.023968,-0.979725,-5.986560,-5.756070,-5.031605,Assault,Assault,Heavy,Scout,Assault,Assault,Heavy,Scout
1,2,False,Map05,Domination,0,#FF0000,#00FF00,True,True,60,0,60,60,140,60,100,100,-5.567025,-0.101519,-0.355085,-7.342392,-3.011566,0.380839,3.170114,6.472121,1.275000,11.982550,-10.915156,4.236153,1.018507,6.970410,-3.966517,-7.417881,Scout,Scout,Scout,Scout,Heavy,Scout,Assault,Assault
2,3,False,Map08,Devastator,0,#FF0000,#00FF00,True,True,70,140,140,0,0,0,100,46,-7.465430,-4.036615,10.611968,0.870345,-5.867641,-5.825762,-1.137856,12.664976,-6.357566,3.520604,-13.028067,-12.942177,-10.912375,4.003931,-11.172128,-7.272110,Support,Heavy,Heavy,Heavy,Assault,Assault,Assault,Assault
3,4,False,Map17,Domination,1,#FF0000,#00FF00,True,True,0,0,0,100,0,0,100,140,-7.163575,12.378940,0.610483,9.084709,0.112198,-10.120893,-7.943638,-14.639385,5.413563,6.929533,-0.110481,5.084709,0.089166,-2.091748,-5.143735,-0.577571,Assault,Assault,Assault,Assault,Scout,Support,Assault,Heavy
4,5,True,Map22,Domination,0,#FF0000,#00FF00,True,True,60,140,60,140,20,100,100,100,-5.403627,-8.297834,9.850985,-10.421861,-10.393658,-0.311284,8.106512,5.091607,-6.588476,-3.373036,-0.156573,3.173592,-3.083063,6.125000,2.914917,2.118116,Scout,Heavy,Scout,Heavy,Heavy,Assault,Assault,Assault


In [5]:
print(game_states_data_training['frag'].value_counts())
print(game_states_data_training['game_mode'].value_counts())
print(game_states_data_training['moving_player'].value_counts())

print(game_states_data_test['frag'].value_counts())
print(game_states_data_test['game_mode'].value_counts())
print(game_states_data_test['moving_player'].value_counts())

False    40410
True      9590
Name: frag, dtype: int64
Domination    29660
Devastator    20340
Name: game_mode, dtype: int64
0    25696
1    24304
Name: moving_player, dtype: int64
Series([], Name: frag, dtype: int64)
Domination    5861
Devastator    4139
Name: game_mode, dtype: int64
0    5069
1    4931
Name: moving_player, dtype: int64


In [6]:
game_states_data_training.describe()

,instance_id,moving_player,hp_player0_unit1,hp_player0_unit2,hp_player0_unit3,hp_player0_unit4,hp_player1_unit1,hp_player1_unit2,hp_player1_unit3,hp_player1_unit4,position_x_player0_unit1,position_x_player0_unit2,position_x_player0_unit3,position_x_player0_unit4,position_x_player1_unit1,position_x_player1_unit2,position_x_player1_unit3,position_x_player1_unit4,position_y_player0_unit1,position_y_player0_unit2,position_y_player0_unit3,position_y_player0_unit4,position_y_player1_unit1,position_y_player1_unit2,position_y_player1_unit3,position_y_player1_unit4
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,25000.500000,0.486080,64.395060,64.610480,85.960020,55.681700,65.918760,67.038240,86.848440,58.812360,-1.913848,0.779050,-0.777698,1.020953,1.032940,3.615924,2.874194,3.718101,-0.264370,0.855062,-0.221130,-1.077481,0.691452,-0.155636,-1.428473,-1.188604
std,14433.901067,0.499811,46.840244,47.237522,52.344052,43.683389,45.830947,46.216319,51.613061,43.062093,6.908333,7.594172,7.591717,7.815768,6.560436,7.102644,6.623827,7.366739,6.525855,6.398245,6.437146,6.527839,7.008903,7.019888,7.199114,7.004185
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-18.189234,-18.126446,-18.604462,-20.205133,-18.590970,-19.396637,-19.602653,-20.352852,-16.725710,-16.671610,-16.456327,-16.711794,-16.697412,-17.287556,-17.690884,-17.714697
25%,12500.750000,0.000000,0.000000,0.000000,60.000000,0.000000,6.000000,4.000000,60.000000,5.000000,-7.122645,-4.706121,-7.029804,-5.361141,-3.764173,-0.545678,-1.203205,-0.790837,-4.443597,-3.738889,-4.909196,-6.711933,-4.675685,-5.358068,-7.297277,-6.682206
50%,25000.500000,0.000000,70.000000,70.000000,100.000000,60.000000,70.000000,70.000000,100.000000,60.000000,-3.037777,-0.071294,-1.344546,0.716799,1.652753,4.840132,3.851756,4.902444,0.184329,0.955398,0.012804,-0.586075,1.083495,0.404670,-1.235838,-1.358622
75%,37500.250000,1.000000,100.000000,100.000000,140.000000,70.000000,100.000000,100.000000,140.000000,94.000000,2.821763,5.435797,5.390935,7.166426,6.187472,8.419000,7.826073,8.891031,4.260918,4.977633,4.284558,3.731799,6.500000,5.319068,4.439662,4.259862
max,50000.000000,1.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,18.969131,20.744686,19.608790,21.548782,18.976147,18.970566,18.935515,22.510527,18.482018,18.793589,18.052328,16.994791,18.044231,18.742983,17.296816,18.541710


In [7]:
game_states_data_test.describe()

,instance_id,frag,moving_player,hp_player0_unit1,hp_player0_unit2,hp_player0_unit3,hp_player0_unit4,hp_player1_unit1,hp_player1_unit2,hp_player1_unit3,hp_player1_unit4,position_x_player0_unit1,position_x_player0_unit2,position_x_player0_unit3,position_x_player0_unit4,position_x_player1_unit1,position_x_player1_unit2,position_x_player1_unit3,position_x_player1_unit4,position_y_player0_unit1,position_y_player0_unit2,position_y_player0_unit3,position_y_player0_unit4,position_y_player1_unit1,position_y_player1_unit2,position_y_player1_unit3,position_y_player1_unit4
count,10000.00000,0.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,55000.50000,NaN,0.493100,63.652800,64.587000,86.464100,55.402800,64.200900,67.286900,86.235000,58.903700,-1.877465,0.773935,-0.561378,1.334828,1.102203,3.749876,2.913107,3.771153,-0.115404,0.857997,-0.309735,-0.967715,0.761339,0.086915,-1.396968,-0.772683
std,2886.89568,NaN,0.499977,47.773942,48.209912,52.237529,44.613296,45.630746,46.997194,52.619565,43.477388,6.930699,7.625565,7.628924,7.728243,6.696973,6.959662,6.488164,7.365589,6.307688,6.366522,6.487798,6.557180,6.986840,6.903939,7.298934,7.070713
min,50001.00000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-16.544956,-18.122234,-18.124960,-18.044678,-18.341543,-17.875000,-18.883690,-20.090740,-16.487595,-15.858482,-16.722094,-16.724260,-16.186256,-16.220690,-17.206245,-17.208950
25%,52500.75000,NaN,0.000000,0.000000,0.000000,60.000000,0.000000,0.000000,8.000000,60.000000,0.000000,-7.086421,-4.661408,-6.894203,-5.026492,-3.784969,-0.387788,-1.085007,-0.829198,-4.177029,-3.818068,-5.230386,-6.479919,-4.687416,-5.215528,-7.475000,-6.236832
50%,55000.50000,NaN,0.000000,70.000000,70.000000,100.000000,60.000000,70.000000,70.000000,100.000000,60.000000,-3.011155,-0.027676,-1.099039,1.640239,1.930721,4.897627,3.874751,4.994250,0.232755,0.866185,-0.012990,-0.643422,1.191882,0.584275,-1.207970,-0.671052
75%,57500.25000,NaN,1.000000,100.000000,100.000000,140.000000,70.000000,100.000000,100.000000,140.000000,100.000000,2.692228,5.390310,5.569179,7.362357,6.247814,8.421898,7.658553,8.957903,4.046183,4.876832,4.247876,3.796300,6.500000,5.575662,4.771732,4.656846
max,60000.00000,NaN,1.000000,1000.000000,1000.000000,1000.000000,1000.000000,140.000000,140.000000,140.000000,140.000000,18.880632,18.970251,18.914967,20.110323,18.954590,18.967615,18.929384,19.208889,17.148480,18.524977,16.543468,16.497812,16.025000,17.288607,17.753927,16.624983


In [283]:
# dropping features without much added value
# engineering new features

X = game_states_data_training

# SKIP 1-40000
# X = X[X['instance_id'] > 40000]

# SKIP 10001-11000
# X = X[(X['instance_id'] < 10000) | (X['instance_id'] >= 11000)]

# SKIP 40001-50000
# X = X[X['instance_id'] <= 40000]

y = X['frag'].astype(int)
y_prepared = y.copy()

# basic columns to drop
X = X.drop('frag', axis=1)
X = X.drop('instance_id', axis=1)

X_player0 = X[[c for c in X.columns if 'player0' in c]]
X_player1 = X[[c for c in X.columns if 'player1' in c]]

# engineering new features
player0_hp_columns = [c for c in X.columns if 'hp_player0' in c]
player1_hp_columns = [c for c in X.columns if 'hp_player1' in c]

X['player0_unit1_alive'] = X.apply(lambda x: x['hp_player0_unit1'] > 0, axis=1)
X['player0_unit2_alive'] = X.apply(lambda x: x['hp_player0_unit2'] > 0, axis=1)
X['player0_unit3_alive'] = X.apply(lambda x: x['hp_player0_unit3'] > 0, axis=1)
X['player0_unit4_alive'] = X.apply(lambda x: x['hp_player0_unit4'] > 0, axis=1)
X['player1_unit1_alive'] = X.apply(lambda x: x['hp_player1_unit1'] > 0, axis=1)
X['player1_unit2_alive'] = X.apply(lambda x: x['hp_player1_unit2'] > 0, axis=1)
X['player1_unit3_alive'] = X.apply(lambda x: x['hp_player1_unit3'] > 0, axis=1)
X['player1_unit4_alive'] = X.apply(lambda x: x['hp_player1_unit4'] > 0, axis=1)

X['player0_alive_units'] = X[player0_hp_columns].apply(lambda x: (x > 0).sum(), axis=1)
X['player1_alive_units'] = X[player1_hp_columns].apply(lambda x: (x > 0).sum(), axis=1)


def number_of_such_units(row, player, unit_type):
    return sum([((row[f'player{player}_unit{i}_type'] == unit_type) and row[f'player{player}_unit{i}_alive']) for i in range(1, 5)])


X['player0_no_alive_assault'] = X.apply(lambda x: number_of_such_units(x, 0, 'Assault'), axis=1)
X['player0_no_alive_heavy'] = X.apply(lambda x: number_of_such_units(x, 0, 'Heavy'), axis=1)
X['player0_no_alive_scout'] = X.apply(lambda x: number_of_such_units(x, 0, 'Scout'), axis=1)
X['player0_no_alive_support'] = X.apply(lambda x: number_of_such_units(x, 0, 'Support'), axis=1)
X['player1_no_alive_assault'] = X.apply(lambda x: number_of_such_units(x, 1, 'Assault'), axis=1)
X['player1_no_alive_heavy'] = X.apply(lambda x: number_of_such_units(x, 1, 'Heavy'), axis=1)
X['player1_no_alive_scout'] = X.apply(lambda x: number_of_such_units(x, 1, 'Scout'), axis=1)
X['player1_no_alive_support'] = X.apply(lambda x: number_of_such_units(x, 1, 'Support'), axis=1)

# further columns to drop
# X = X.drop(['player0_colour', 'player1_colour'], axis=1)

print(len(X.columns))

57


In [284]:
def unit_dist(row, unit_no_p0, unit_no_p1):
    return math.dist(
        [row[f'position_x_player0_unit{unit_no_p0}'], row[f'position_y_player0_unit{unit_no_p0}']], 
        [row[f'position_x_player1_unit{unit_no_p1}'], row[f'position_y_player1_unit{unit_no_p1}']])

for i in range(1, 5):
    for j in range(1, 5):
        X[f'p0u{i}_p1u{j}_dist'] = X.apply(lambda x: unit_dist(x, i, j), axis=1)
        
print(len(X.columns))

73


In [285]:
# transforming X and y

numerical_columns = [c for c in X.columns if (X[c].dtype == 'bool' or X[c].dtype == 'float64' or X[c].dtype == 'int64')]
categorical_columns = [c for c in X.columns if X[c].dtype == 'object']

ct = make_column_transformer(
    (StandardScaler(), numerical_columns),
    (OneHotEncoder(), categorical_columns),
    remainder='drop')

X_t = ct.fit_transform(X)

transformed_X = pd.DataFrame(
    X_t,
    columns=ct.get_feature_names_out(X.columns)
)

X = transformed_X.copy()

X_prepared = X.copy()

X.shape

(50000, 135)

In [199]:
gbm_param_grid = {
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'learning_rate': [0.1, 0.2, 0.3, 0.4],
    'max_depth': [2, 3, 4, 5, 6, 7],
    'min_child_weight': [0, 1, 2, 3, 4, 5, 6, 7, 8],
    'n_estimators': [50, 100, 200, 300, 400, 500],
    'scale_pos_weight': [1, 2, 3, 4],
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1]
}

In [200]:
# # MODEL FOR PLAYER0

# y0 = y[X['standardscaler__moving_player'] < 0]
# X0 = X[X['standardscaler__moving_player'] < 0]
# X0 = X0.drop('standardscaler__moving_player', axis=1)

# # IMPORTANT - DROPPING FEATURES
# X0 = X0.drop([c for c in X0.columns if (('unit' in c) and ('alive' in c) and ('units' not in c))], axis=1)

# print(X0.shape)
# print(y0.shape)

# x_train, x_test, y_train, y_test = train_test_split(X0, y0, test_size=0.2, random_state=42)

# gbm = xgb.XGBClassifier()

# # set up RandomizedSearchCV()
# randomized_mse_player0 = RandomizedSearchCV(estimator=gbm, 
#                                             param_distributions=gbm_param_grid,
#                                             scoring='roc_auc',
#                                             cv=3,
#                                             n_iter=3,
#                                             verbose=4,
#                                             return_train_score=True,
#                                             random_state=420)      

# # fit the models
# randomized_mse_player0.fit(x_train, y_train)               

# print("Best parameters found: ", randomized_mse_player0.best_params_)

# # for el in zip(X0.columns, randomized_mse_player0.best_estimator_.feature_importances_):
# #     print(el)

# y_pred = randomized_mse_player0.predict_proba(x_test)[:, 1]

# print(np.mean(y_pred))
# print(np.median(y_pred))
# print(np.std(y_pred))

# print(f'\nTest score: {roc_auc_score(y_test, y_pred)}')

In [201]:
# # MODEL FOR PLAYER1

# y1 = y[X['standardscaler__moving_player'] > 0]
# X1 = X[X['standardscaler__moving_player'] > 0]
# X1 = X1.drop('standardscaler__moving_player', axis=1)

# # IMPORTANT - DROPPING FEATURES
# X1 = X1.drop([c for c in X1.columns if (('unit' in c) and ('alive' in c) and ('units' not in c))], axis=1)

# print(X1.shape)
# print(y1.shape)

# x_train, x_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2)

# gbm = xgb.XGBClassifier()

# # set up RandomizedSearchCV()
# randomized_mse_player1 = RandomizedSearchCV(estimator=gbm, 
#                                             param_distributions=gbm_param_grid,
#                                             scoring='roc_auc',
#                                             cv=3,
#                                             n_iter=3,
#                                             verbose=4,
#                                             return_train_score=True)      

# # fit the model
# randomized_mse_player1.fit(x_train, y_train)               

# print("Best parameters found: ", randomized_mse_player1.best_params_)

# # for el in zip(X0.columns, randomized_mse_player1.best_estimator_.feature_importances_):
# #     print(el)

# y_pred = randomized_mse_player1.predict_proba(x_test)[:, 1]

# print(np.mean(y_pred))
# print(np.median(y_pred))
# print(np.std(y_pred))

# print(f'\nTest score: {roc_auc_score(y_test, y_pred)}')

In [202]:
# # MODEL FOR BOTH PLAYERS (RANDOMIZEDSEARCHCV)

# print(X.shape)

# # FEATURE REMOVAL

# X_f = X.copy()
# # X_f = X_f.drop([c for c in X_f.columns if 'position' in c], axis=1)
# X_f = X_f.drop([c for c in X_f.columns if (('unit' in c) and ('alive' in c) and ('units' not in c))], axis=1)

# print(X_f.shape)

# x_train, x_test, y_train, y_test = train_test_split(X_f, y, test_size=0.2)

# gbm = xgb.XGBClassifier()

# # set up RandomizedSearchCV()
# randomized_mse = RandomizedSearchCV(estimator=gbm, 
#                         param_distributions=gbm_param_grid,
#                         scoring='roc_auc',
#                         cv=3,
#                         n_iter=3,
#                         verbose=4,
#                         return_train_score=True)      

# # fit the model
# randomized_mse.fit(x_train, y_train)               

# print("Best parameters found: ", randomized_mse.best_params_)

# y_pred = randomized_mse.predict_proba(x_test)[:, 1]

# print(np.mean(y_pred))
# print(np.median(y_pred))
# print(np.std(y_pred))

# print(f'\nTest score: {roc_auc_score(y_test, y_pred)}')

In [333]:
# FINAL MODEL

X_m = X_prepared.copy()

X_m = X_m.drop([c for c in X_m.columns if 'map' in c], axis=1)
# X_m = X_m.drop([c for c in X_m.columns if 'position' in c], axis=1)
# X_m = X_m.drop([c for c in X_m.columns if '_type_' in c], axis=1)
# X_m = X_m.drop([c for c in X_m.columns if '_no_alive_' in c], axis=1)

# img_model_preds = []
# with open('img_pred_40001_50000.txt', 'r') as f:
#     for line in f:
#         img_model_preds.append(float(line[:-1]))
# X_m['img_model'] = img_model_preds

print(X_m.shape)

x_train, x_test, y_train, y_test = train_test_split(X_m, y_prepared, test_size=0.1)

model1 = xgb.XGBClassifier(colsample_bytree=1,
                           # eval_metric='auc',
                           gamma=0.1, 
                           learning_rate=0.1, 
                           max_depth=6, 
                           min_child_weight=250, 
                           n_estimators=400, 
                           scale_pos_weight=1, 
                           subsample=1)

# model1.fit(x_train, y_train)
model1.fit(x_train, y_train, early_stopping_rounds=20, eval_metric='auc', eval_set=[(x_test, y_test)], verbose=True)

y_train_pred = model1.predict_proba(x_train)[:, 1]

print(f'\nTrain score: {roc_auc_score(y_train, y_train_pred)}\n')

y_pred = model1.predict_proba(x_test)[:, 1]

print(np.mean(y_pred))
print(np.median(y_pred))
print(np.std(y_pred))

print(f'\nTest score: {roc_auc_score(y_test, y_pred)}')

final_model = model1

# for el in zip(X_m.columns, model1.feature_importances_):
#     print(el)

(50000, 111)


C:\Users\micha\Anaconda3\envs\ml\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\micha\Anaconda3\envs\ml\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.64894
[1]	validation_0-auc:0.66716
[2]	validation_0-auc:0.67039
[3]	validation_0-auc:0.67613
[4]	validation_0-auc:0.68351
[5]	validation_0-auc:0.68525
[6]	validation_0-auc:0.68739
[7]	validation_0-auc:0.68753
[8]	validation_0-auc:0.68982
[9]	validation_0-auc:0.69281
[10]	validation_0-auc:0.69707
[11]	validation_0-auc:0.69747
[12]	validation_0-auc:0.69899
[13]	validation_0-auc:0.69835
[14]	validation_0-auc:0.69951
[15]	validation_0-auc:0.69993
[16]	validation_0-auc:0.70224
[17]	validation_0-auc:0.70368
[18]	validation_0-auc:0.70356
[19]	validation_0-auc:0.70495
[20]	validation_0-auc:0.70489
[21]	validation_0-auc:0.70571
[22]	validation_0-auc:0.70595
[23]	validation_0-auc:0.70587
[24]	validation_0-auc:0.70715
[25]	validation_0-auc:0.70839
[26]	validation_0-auc:0.71025
[27]	validation_0-auc:0.71068
[28]	validation_0-auc:0.71193
[29]	validation_0-auc:0.71289
[30]	validation_0-auc:0.71578
[31]	validation_0-auc:0.71704
[32]	validation_0-auc:0.71828
[33]	validation_0-au

In [334]:
# OFFICIAL TEST SET PREDICTION
# modify the test data the same way the training data was modified and predict results

# TEST ON TEST SET
X = game_states_data_test

# TEST ON 10001-11000
# X = game_states_data_training
# X = X[(X['instance_id'] > 10000) & (X['instance_id'] <= 11000)]
# y_test = X['frag'].astype(int)

# TEST ON 40001-50000
# X = game_states_data_training
# X = X[X['instance_id'] > 40000]
# y_test = X['frag'].astype(int)


# basic columns to drop
X = X.drop('frag', axis=1)
X = X.drop('instance_id', axis=1)

# engineering new features
X_player0 = X[[c for c in X.columns if 'player0' in c]]
X_player1 = X[[c for c in X.columns if 'player1' in c]]

player0_hp_columns = [c for c in X.columns if 'hp_player0' in c]
player1_hp_columns = [c for c in X.columns if 'hp_player1' in c]

X['player0_unit1_alive'] = X.apply(lambda x: x['hp_player0_unit1'] > 0, axis=1)
X['player0_unit2_alive'] = X.apply(lambda x: x['hp_player0_unit2'] > 0, axis=1)
X['player0_unit3_alive'] = X.apply(lambda x: x['hp_player0_unit3'] > 0, axis=1)
X['player0_unit4_alive'] = X.apply(lambda x: x['hp_player0_unit4'] > 0, axis=1)
X['player1_unit1_alive'] = X.apply(lambda x: x['hp_player1_unit1'] > 0, axis=1)
X['player1_unit2_alive'] = X.apply(lambda x: x['hp_player1_unit2'] > 0, axis=1)
X['player1_unit3_alive'] = X.apply(lambda x: x['hp_player1_unit3'] > 0, axis=1)
X['player1_unit4_alive'] = X.apply(lambda x: x['hp_player1_unit4'] > 0, axis=1)

X['player0_alive_units'] = X[player0_hp_columns].apply(lambda x: (x > 0).sum(), axis=1)
X['player1_alive_units'] = X[player1_hp_columns].apply(lambda x: (x > 0).sum(), axis=1)


def number_of_such_units(row, player, unit_type):
    return sum([((row[f'player{player}_unit{i}_type'] == unit_type) and row[f'player{player}_unit{i}_alive']) for i in range(1, 5)])


X['player0_no_alive_assault'] = X.apply(lambda x: number_of_such_units(x, 0, 'Assault'), axis=1)
X['player0_no_alive_heavy'] = X.apply(lambda x: number_of_such_units(x, 0, 'Heavy'), axis=1)
X['player0_no_alive_scout'] = X.apply(lambda x: number_of_such_units(x, 0, 'Scout'), axis=1)
X['player0_no_alive_support'] = X.apply(lambda x: number_of_such_units(x, 0, 'Support'), axis=1)
X['player1_no_alive_assault'] = X.apply(lambda x: number_of_such_units(x, 1, 'Assault'), axis=1)
X['player1_no_alive_heavy'] = X.apply(lambda x: number_of_such_units(x, 1, 'Heavy'), axis=1)
X['player1_no_alive_scout'] = X.apply(lambda x: number_of_such_units(x, 1, 'Scout'), axis=1)
X['player1_no_alive_support'] = X.apply(lambda x: number_of_such_units(x, 1, 'Support'), axis=1)


def unit_dist(row, unit_no_p0, unit_no_p1):
    return math.dist(
        [row[f'position_x_player0_unit{unit_no_p0}'], row[f'position_y_player0_unit{unit_no_p0}']], 
        [row[f'position_x_player1_unit{unit_no_p1}'], row[f'position_y_player1_unit{unit_no_p1}']])


for i in range(1, 5):
    for j in range(1, 5):
        X[f'p0u{i}_p1u{j}_dist'] = X.apply(lambda x: unit_dist(x, i, j), axis=1)

# further columns to drop before transformation (but after feature engineering!)
# X = X.drop(['player0_colour', 'player1_colour'], axis=1)
        
# transforming X and y
numerical_columns = [c for c in X.columns if (X[c].dtype == 'bool' or X[c].dtype == 'float64' or X[c].dtype == 'int64')]
categorical_columns = [c for c in X.columns if X[c].dtype == 'object']

X_t = ct.transform(X)

transformed_X = pd.DataFrame(
    X_t,
    columns=ct.get_feature_names_out(X.columns)
)

X = transformed_X.copy()

# further columns to drop after transformation
X = X.drop([c for c in X.columns if 'map' in c], axis=1)
# X = X.drop([c for c in X.columns if 'position' in c], axis=1)
# X = X.drop([c for c in X.columns if '_type_' in c], axis=1)
# X = X.drop([c for c in X.columns if '_no_alive_' in c], axis=1)

y_pred = final_model.predict_proba(X)[:, 1]

# with open('res_10001_11000.txt', 'w') as f:
#     for y in y_pred:
#         f.write(f'{y}\n')
        
print(np.mean(y_pred))
print(np.median(y_pred))
print(np.std(y_pred))

print(len(y_test))
print(len(y_pred))

print(f'\nTest score: {roc_auc_score(y_test, y_pred)}')

0.19374768
0.16206726
0.1464716
5000
10000


ValueError: Found input variables with inconsistent numbers of samples: [5000, 10000]

In [335]:
final_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.1, gpu_id=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=250, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [336]:
# LOAD OTHER RESULTS MERGE AND OUT

y_main_model = y_pred
y_second_model = []

with open('img_screens_test_res.txt', 'r') as f:
    for line in f:
        y_second_model.append(float(line[:-1]))

# for ratio in range(1, 26):
#     y_unified_model = [((ratio - 1) * y1 + y2) / ratio for (y1, y2) in zip(y_main_model, y_second_model)]
#     print(f'Non-scaled dumb {ratio}x average score "train": {roc_auc_score(y_test, y_unified_model)}')  # 13
        
ratio = 13
y_unified_model = [((ratio - 1) * y1 + y2) / ratio for (y1, y2) in zip(y_main_model, y_second_model)]

with open('unified_res.txt', 'w') as f:
    for y in y_unified_model:
        f.write(f'{y}\n')

In [93]:
min(y_unified_model)

0.003000051284638735

In [63]:
y_main_model = y_pred
y_second_model = []

with open('img_pred_40001_50000.txt', 'r') as f:
    for line in f:
        y_second_model.append(float(line[:-1]))

y_main_model_scaled = MinMaxScaler().fit_transform(y_main_model.reshape(-1, 1))
y_second_model_scaled = MinMaxScaler().fit_transform(np.array(y_second_model).reshape(-1, 1))

y_main_model_scaled = [y for l in y_main_model_scaled for y in l]
y_second_model_scaled = [y for l in y_second_model_scaled for y in l]

# print(y_main_model[:10])
# print(y_main_model_scaled[:10])
# print(y_second_model[:10])
# print(y_second_model_scaled[:10])

y_unified_model = [(9 * y1 + 1 * y2) / 10 for (y1, y2) in zip(y_main_model, y_second_model)]
y_unified_model_scaled = [(9 * y1 + 1 * y2) / 10 for (y1, y2) in zip(y_main_model_scaled, y_second_model_scaled)]

print(f'\nTest 1 score: {roc_auc_score(y_test, y_main_model)}')
print(f'\nTest 2 score: {roc_auc_score(y_test, y_second_model)}')
print(f'\nTest 3 score: {roc_auc_score(y_test, y_unified_model)}')
print(f'\nTest 3 score: {roc_auc_score(y_test, y_unified_model_scaled)}')


Test 1 score: 0.7623761909310954

Test 2 score: 0.6225310757823442

Test 3 score: 0.763992904647954

Test 3 score: 0.7640776325934991


In [73]:
split = 8000
X_meta_train = list(zip(y_main_model[:split], y_second_model[:split]))
X_meta_test = list(zip(y_main_model[split:], y_second_model[split:]))
y_meta_train = y_test[:split]
y_meta_test = y_test[split:]

reg = LinearRegression().fit(X_meta_train, y_meta_train)

print(reg.coef_)
print(reg.intercept_)

y_reg_train_pred = reg.predict(X_meta_train)
y_reg_test_pred = reg.predict(X_meta_test)
y_reg_test_pred = [(y + reg.intercept_ + 0.001) / max(y_reg_test_pred) for y in y_reg_test_pred]

print(f'Train score: {roc_auc_score(y_meta_train, y_reg_train_pred)}')
print(f'Test score: {roc_auc_score(y_meta_test, y_reg_test_pred)}')

[0.98794314 0.09521972]
-0.03894585523249547
Train score: 0.7662622787223425
Test score: 0.75492793419967


In [74]:
gbm = xgb.XGBClassifier()

gbm_param_grid = {
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
    'gamma': [0.1, 0.2, 0.3, 0.4],
    'learning_rate': [0.1, 0.2, 0.3, 0.4],
    'max_depth': [2, 3, 4, 5, 6],
    'min_child_weight': [0, 25, 50, 100],
    'n_estimators': [100, 200, 300, 400, 500, 600],
    'subsample': [0.6, 0.7, 0.8, 0.9]
}

meta_model = RandomizedSearchCV(estimator=gbm, 
                                            param_distributions=gbm_param_grid,
                                            scoring='roc_auc',
                                            cv=4,
                                            n_iter=5,
                                            verbose=4,
                                            return_train_score=True,
                                            random_state=420)      

meta_model.fit(X_meta_train, y_meta_train)

print("Best parameters found: ", meta_model.best_params_)

y_meta_train_pred = meta_model.predict_proba(X_meta_train)[:, 1]

print(f'\nTrain score: {roc_auc_score(y_meta_train, y_meta_train_pred)}')

y_meta_test_pred = meta_model.predict_proba(X_meta_test)[:, 1]

print(f'Test score: {roc_auc_score(y_meta_test, y_meta_test_pred)}')

Fitting 4 folds for each of 5 candidates, totalling 20 fits
[CV 1/4] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.4, max_depth=3, min_child_weight=50, n_estimators=100, subsample=0.7;, score=(train=0.797, test=0.760) total time=   1.0s
[CV 2/4] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.4, max_depth=3, min_child_weight=50, n_estimators=100, subsample=0.7;, score=(train=0.800, test=0.752) total time=   0.9s
[CV 3/4] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.4, max_depth=3, min_child_weight=50, n_estimators=100, subsample=0.7;, score=(train=0.799, test=0.762) total time=   0.8s
[CV 4/4] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.4, max_depth=3, min_child_weight=50, n_estimators=100, subsample=0.7;, score=(train=0.802, test=0.745) total time=   0.8s
[CV 1/4] END colsample_bytree=0.7, gamma=0.3, learning_rate=0.4, max_depth=6, min_child_weight=25, n_estimators=200, subsample=0.9;, score=(train=0.859, test=0.733) total time=   2.6s
[CV 2/4] END colsamp

In [77]:
print(f'Original model score: {roc_auc_score(y_meta_test, y_main_model_scaled[split:])}\n')

print(f'Regression score: {roc_auc_score(y_meta_test, y_reg_test_pred)}')
print(f'CV-XGBoost score: {roc_auc_score(y_meta_test, y_meta_test_pred)}')
print(f'Dumb 9/1 average score: {roc_auc_score(y_meta_test, y_unified_model[split:])}')

Original model score: 0.7542817270008251

Regression score: 0.75492793419967
CV-XGBoost score: 0.7486640753919143
Dumb 9/1 average score: 0.7547796771864688


In [81]:
for ratio in range(1, 26):
    y_unified_model = [((ratio - 1) * y1 + y2) / ratio for (y1, y2) in zip(y_main_model, y_second_model)]
    print(f'Non-scaled dumb {ratio}x average score "train": {roc_auc_score(y_test, y_unified_model)}')  # 13

Non-scaled dumb 1x average score "train": 0.6225310757823442
Non-scaled dumb 2x average score "train": 0.7171583298595618
Non-scaled dumb 3x average score "train": 0.7456980731928364
Non-scaled dumb 4x average score "train": 0.7556964806006143
Non-scaled dumb 5x average score "train": 0.7598815758875294
Non-scaled dumb 6x average score "train": 0.7618664849450327
Non-scaled dumb 7x average score "train": 0.7629068000883796
Non-scaled dumb 8x average score "train": 0.7634741809828429
Non-scaled dumb 9x average score "train": 0.7638298535395092
Non-scaled dumb 10x average score "train": 0.763992904647954
Non-scaled dumb 11x average score "train": 0.7640801817607213
Non-scaled dumb 12x average score "train": 0.7640850251784433
Non-scaled dumb 13x average score "train": 0.7641005750984982
Non-scaled dumb 14x average score "train": 0.7640646318406664
Non-scaled dumb 15x average score "train": 0.7640403510228756
Non-scaled dumb 16x average score "train": 0.764004662681766
Non-scaled dumb 17x